### Search Engine Tools and Agents

%pip install arxiv


%pip install wikipedia


%pip install groq

In [23]:
## Tools Creation
## Arxiv -- Research
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

Wrapper around WikipediaAPI.

To use, you should have the wikipedia python package installed. This wrapper will use the Wikipedia API to conduct searches and fetch page summaries. By default, it will return the page summaries of the top-k results. It limits the Document content by doc_content_chars_max.

In [24]:
## Used the Inbuilt tools of the Wikipedia

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [25]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [26]:
tools=[wiki,arxiv]

In [27]:
##Using Custom tools[RAG Tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [28]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x329c01290>, search_kwargs={})

In [29]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search",  "Search any information about Langsmith")

retriever_tool.name

'langsmith-search'

In [30]:
tools = [wiki,arxiv,retriever_tool]

In [31]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/surya/.pyenv/versions/3.11.0/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10ba7cf40>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x329c01290>, sear

## Video - 2
### Executing Tools And LLM with Agent Executors

## Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [47]:
from dotenv import load_dotenv
import os

load_dotenv()

from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults

# Load API Keys
tavily_api_key = os.getenv("TAVILY_API_KEY")

if not tavily_api_key:
    print("Error: TAVILY_API_KEY is missing. Check your .env file.")
    exit(1)

# Initialize Ollama Model (No API Key Required)
try:
    llm = Ollama(model="mistral")  # You can also use "llama3"
except Exception as e:
    print(f"Error initializing Ollama LLM: {str(e)}")
    exit(1)

# Function to Generate Text using Ollama
def generate_text(prompt: str):
    try:
        response = llm.invoke(prompt)
        return response
    except Exception as e:
        print(f"Error generating text: {str(e)}")
        return "Error generating text."

# Define Ollama Tool
text_gen_tool = Tool(
    name="ollama_textgen",
    func=generate_text,
    description="Generates text using Ollama (Mistral model)"
)

# Load Tavily Search Tool
try:
    search_tool = TavilySearchResults(api_key=tavily_api_key)
except Exception as e:
    print(f"Error loading Tavily Search Tool: {str(e)}")
    exit(1)

# Combine Tools
tools = [text_gen_tool, search_tool]

# Create an Agent
try:
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
except Exception as e:
    print(f"Error creating agent: {str(e)}")
    exit(1)

# Run the Agent
try:
    response = agent.invoke({"input": "calculate SBI Loan Amount of 5000000/-, with 11.60% to 14.10% per annum intrest, and How much that I have to pay per month, and Time limit is 10 years"})
    if response:
        print("\nAI Response:\n", response)
    else:
        print("No response received from the agent.")
except Exception as e:
    print(f"Error running agent: {str(e)}")




> Entering new AgentExecutor chain...
 To answer this question, I need to calculate the loan amount, interest rate, monthly payment, and time period provided. I will use the ollama_textgen tool to structure the problem mathematically, then perform calculations using standard financial formulas. After that, I'll determine the monthly payment.

Action: ollama_textgen
Action Input: "Let's calculate the SBI Loan Amount, Monthly Payment, and Time Period given the principal amount of 5000000/-, Interest Rate Range (11.60% - 14.10% per annum), and Time Limit is 10 years."
Observation:  To calculate the SBI Loan details, we'll use the formula for a loan with constant monthly payments (amortizing loan). The formula is:

Loan Payment (P) = [P[r(1+r)^n] / ((1+r)^n – 1)] * L

Where:
- P is the periodic payment.
- r is the monthly interest rate (annual interest rate / 12).
- n is the total number of periods.
- L is the loan amount (principal).

First, let's find the monthly interest rate for both